# Dataset preprocessing

The goal of this notebook is to create a preprocessed kaggle dataset out of the competition dataset.  
For now, the preprocessing will be based on [this notebook](https://www.kaggle.com/code/vonmainstein/imu-tof).  
It consists of the following steps:
-   Set the appropriate dtypes (helps with RAM usage).
-   Impute missing feature values with forward, backward and then 0 filling.
-   Split the dataset into multiple cross validation folds.
-   Standardize feature values.
-   Pad/Truncate the sequences to the same length.  

> Note:  
> - Demographics data set will be ignored for now.  

## Imports

In [1]:
import os
import json
from os.path import join
from tqdm.notebook import tqdm
from itertools import repeat, starmap

import numpy as np
import pandas as pd
from numpy import ndarray
import plotly.express as px
from pandas import DataFrame as DF
from scipy.spatial.transform import Rotation
from kagglehub import whoami, competition_download, dataset_upload

from config import *

## Data preprocessing

### Load dataset
Requires to be logged in if this notebook is not running on kaggle, go to [your settings](https://www.kaggle.com/settings) to create an access token and put it in `~/.kaggle/`.

In [ ]:
competition_dataset_path = competition_download(COMPETITION_HANDLE)
df = pd.read_csv(join(competition_dataset_path, "train.csv"), dtype=DATASET_DF_DTYPES)

### Impute missing data
Perform forward, backward and then filling of all NaN sequences.

In [ ]:
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS))
# Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
# So we replace them by NaN and then perform imputing.  
tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

def imputed_features(df:DF) -> DF:
    # Missing ToF values are already imputed by -1 which is inconvinient since we want all missing values to be NaN.    
    # So we replace them by NaN and then perform imputing.  
    tof_vals_to_nan = {col: -1.0 for col in df.columns if col.startswith("tof")}
    fillna_val_per_col = {col: 1.0 if col == 'rot_w' else 0 for col in df.columns}

    df[feature_cols] = (
        df
        .loc[:, feature_cols]
        # df.replace with np.nan sets dtype to floar64 so we set it back to float32
        .replace(tof_vals_to_nan, value=np.nan)
        .astype("float32")
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .ffill()
        .groupby(df["sequence_id"], observed=True, as_index=False)
        .bfill()
        # In case there are only nan in the column in the sequence
        .fillna(fillna_val_per_col)
    )
    return df

df = imputed_features(df)

### Norm quaternions
This allows us to parse quaternions with `scipy.spatial.transform.Rotation`.

In [ ]:
QUATERNION_COLS = ['rot_w', 'rot_x', 'rot_y', 'rot_z']
def norm_quat_rotations(df:DF) -> DF:
    df[QUATERNION_COLS] /= np.linalg.norm(df[QUATERNION_COLS], axis=1, keepdims=True)
    return df

### Add gravity free acceleration features

In [ ]:
GRAVITY_WORLD = np.array([0, 0, 9.81])
ACCELRATION_COLS = ["acc_x", "acc_y", "acc_z"]
GRAVITY_FREE_ACC_COLS = ["gravity_free_" + col for col in ACCELRATION_COLS]
def add_gravity_free_acc_cols(df:DF) -> DF:
    # Vectorized version of https://www.kaggle.com/code/wasupandceacar/lb-0-82-5fold-single-bert-model#Dataset `remove_gravity_from_acc`
    rotations = Rotation.from_quat(df[QUATERNION_COLS])
    gravity_sensor_frame = rotations.apply(GRAVITY_WORLD, inverse=True)
    df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
    return df

df = add_gravity_free_acc_cols(df)

/tmp/ipykernel_8848/3192284712.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
/tmp/ipykernel_8848/3192284712.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[GRAVITY_FREE_ACC_COLS] = df[ACCELRATION_COLS] - gravity_sensor_frame
/tmp/ipykernel_8848/3192284712.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

### Euler angles from quaternions

In [ ]:
EULER_ANGLES_COLS = ["euler_x", "euler_y", "euler_z"]
def rot_euler_angles(seq:DF) -> ndarray:
    try:
        quat_cols = seq[QUATERNION_COLS]
        rotation = Rotation.from_quat(quat_cols)
        euler_data = rotation.as_euler("xyz").squeeze()
        angles_df = DF(
            data=euler_data,
            columns=EULER_ANGLES_COLS
        )
        return angles_df
    except ValueError as e:
        print(quat_cols)
        raise e

rot_euler_angles_df = (
    df
    .groupby("sequence_id", as_index=False, observed=True)
    .apply(rot_euler_angles, include_groups=False)
    .loc[:, EULER_ANGLES_COLS]
    .values
)
display(rot_euler_angles_df)
df[EULER_ANGLES_COLS] = rot_euler_angles_df

array([[ 0.13973522,  0.76901321,  1.06602422],
       [ 0.07560634,  0.75346624,  0.98876692],
       [-0.23884316,  0.6830792 ,  0.69691553],
       ...,
       [ 2.17062915,  0.40229491, -3.09554491],
       [ 2.19296719,  0.42823974, -3.09448917],
       [ 2.19121165,  0.41152166, -3.09799802]], shape=(574945, 3))

/tmp/ipykernel_8848/1959754906.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[EULER_ANGLES_COLS] = rot_euler_angles_df
/tmp/ipykernel_8848/1959754906.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[EULER_ANGLES_COLS] = rot_euler_angles_df
/tmp/ipykernel_8848/1959754906.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

### One hot encode target values.

In [ ]:
one_hot_target = pd.get_dummies(df["gesture"])
df[one_hot_target.columns] = one_hot_target
df

/tmp/ipykernel_8848/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_8848/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[one_hot_target.columns] = one_hot_target
/tmp/ipykernel_8848/1507406683.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Neck - scratch,Text on phone,Wave hello,Write name in air,Write name on leg,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,False,False,False,False,False
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,False,False,False,False,False
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,False,False,False,False,False
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,False,False,False,False,False
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,True,False,False,False,False,False
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,True,False,False,False,False,False
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,True,False,False,False,False,False
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,True,False,False,False,False,False


### ToF data aggregation.
Time of Flight columns take most of the data, let's reduce their size by aggregating by mean for each Time of Flight sensor.

In [ ]:
def agg_tof_cols_per_sensor(df:DF) -> DF:
    for tof_idx in range(1, 6):
        tof_name = f"tof_{tof_idx}"
        tof_cols = [f"{tof_name}_v{v_idx}" for v_idx in range(64)]
        if any(map(lambda col: col not in df.columns, tof_cols)):
            print(f"Some (or) all ToF {tof_idx} columns are not in the df. Maybe you already ran this cell?")
            continue
        df = (
            df
            # Need to use a dict otherwise the name of the col will be "tof_preffix" instead of the value it contains
            .assign(**{tof_name:df[tof_cols].mean(axis="columns")})
            .drop(columns=tof_cols)
        )
    return df

df = agg_tof_cols_per_sensor(df)
# Redifine feature_cols now that there are less of them.
feature_cols = list(set(df.columns) - set(META_DATA_COLUMNS) - set(df["gesture"].unique().tolist()))

df

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,...,Drink from bottle/cup,Pinch knee/leg skin,Pull air toward your face,Scratch knee/leg skin,Glasses on/off,tof_1,tof_2,tof_3,tof_4,tof_5
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,...,False,False,False,False,False,139.250000,117.109375,91.687500,123.359375,135.343750
1,SEQ_000007_000001,Target,SEQ_000007,1,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.949219,...,False,False,False,False,False,139.796875,119.671875,97.921875,124.406250,137.000000
2,SEQ_000007_000002,Target,SEQ_000007,2,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.722656,...,False,False,False,False,False,142.375000,128.359375,116.953125,125.687500,140.234375
3,SEQ_000007_000003,Target,SEQ_000007,3,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.601562,...,False,False,False,False,False,154.109375,142.093750,144.515625,149.078125,142.609375
4,SEQ_000007_000004,Target,SEQ_000007,4,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,5.566406,...,False,False,False,False,False,177.953125,149.453125,161.828125,163.765625,151.265625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531_000048,Non-Target,SEQ_065531,48,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.503906,...,False,False,False,False,False,68.562500,67.750000,144.437500,74.062500,52.843750
574941,SEQ_065531_000049,Non-Target,SEQ_065531,49,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.773438,...,False,False,False,False,False,70.234375,66.656250,144.000000,70.406250,54.531250
574942,SEQ_065531_000050,Non-Target,SEQ_065531,50,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.082031,...,False,False,False,False,False,66.671875,62.906250,136.906250,70.109375,57.468750
574943,SEQ_065531_000051,Non-Target,SEQ_065531,51,SUBJ_039498,Seated Lean Non Dom - FACE DOWN,Performs gesture,Gesture,Write name on leg,3.964844,...,False,False,False,False,False,67.218750,64.500000,140.531250,75.609375,54.937500


### Split into folds

In [ ]:
def split_dataset(df:DF, by="subject") -> tuple[DF, DF]:
    unique_sequences = df[by].unique()
    validation_sequences = pd.Series(unique_sequences).sample(
        frac=VALIDATION_FRACTION, replace=False
    )

    validation_set = df[df[by].isin(validation_sequences)].copy()
    train_set = df[~df[by].isin(validation_sequences)].copy()

    return train_set, validation_set

folds = list(map(split_dataset, repeat(df, N_FOLDS)))

In [ ]:
display(df.shape)
display(folds[0][0].shape)
display(folds[0][1].shape)

(574945, 50)

(463941, 50)

(111004, 50)

In [ ]:
for train_df, validation_df in folds:
    print(train_df.shape, validation_df.shape)

(463941, 50) (111004, 50)
(466501, 50) (108444, 50)
(462781, 50) (112164, 50)
(463671, 50) (111274, 50)
(462226, 50) (112719, 50)


### Std norm
Standard scale the feature cols (should probably do something different for IMU cols).  
<!-- *Deprecated, std norm is now performed at dataset creation to avoid target leakage.*   -->

In [ ]:
def std_norm_dataset(train:DF, val:DF) -> tuple[DF, DF]:
    means = train[feature_cols].mean()
    stds = train[feature_cols].std()
    train.loc[:, feature_cols] = (train[feature_cols] - means) / stds
    val.loc[:, feature_cols] = (val[feature_cols] - means) / stds
    return train, val

folds = list(starmap(std_norm_dataset, folds))

gravity_free_acc_z   -6.861285e-18
rot_w                 3.430643e-18
euler_x              -9.801836e-19
tof_5                -2.940551e-18
tof_2                -5.391010e-18
gravity_free_acc_x    6.861285e-18
acc_z                 9.801836e-18
euler_z              -8.821652e-18
thm_4                 4.900918e-18
thm_2                 5.881101e-18
thm_5                -1.960367e-18
acc_x                -6.861285e-18
thm_1                -3.920734e-18
thm_3                 5.881101e-18
gravity_free_acc_y   -3.920734e-18
tof_1                -3.920734e-18
euler_y               5.391010e-18
rot_z                 5.881101e-18
rot_y                 5.881101e-18
acc_y                -9.311744e-18
tof_4                 6.861285e-18
rot_x                 4.410826e-18
tof_3                 3.675688e-18
dtype: float64

Normalize full dataset.

In [ ]:
# Retain full dataset meta data for inference
full_dataset_meta_data = {
    "mean": df[feature_cols].mean().to_dict(),
    "std": df[feature_cols].std().to_dict(),
}
df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']

/tmp/ipykernel_8848/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.99715055 -0.95773732 -0.63026367 ... -1.08920292 -1.07042212
 -1.09237714]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_8848/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.40467845  0.43377058  0.49058256 ... -0.96320044 -1.007662
 -1.00162401]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  df.loc[:, feature_cols] = (df[feature_cols] - full_dataset_meta_data["mean"]) / full_dataset_meta_data['std']
/tmp/ipykernel_8848/4279755229.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future

Verify the mean and std of the full dataset.

In [ ]:
df[feature_cols].agg(["mean", "std"])

,gravity_free_acc_z,rot_w,euler_x,tof_5,tof_2,gravity_free_acc_x,acc_z,euler_z,thm_4,thm_2,...,thm_3,gravity_free_acc_y,tof_1,euler_y,rot_z,rot_y,acc_y,tof_4,rot_x,tof_3
mean,-2.372822e-18,-6.315844e-08,1.581881e-17,-5.993022e-08,1.687596e-08,2.293728e-17,-2.971537e-09,-4.982926e-17,-1.427808e-07,2.822679e-07,...,1.823665e-07,-8.858535e-17,1.908273e-07,1.581881e-18,1.855411e-08,-8.728876e-10,1.186949e-08,1.404808e-08,-2.663767e-08,2.663846e-09
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


Let's compare the train to validation mean/std skews.

In [ ]:
folds[0][1][feature_cols].mean() #.agg(["mean", "std"]),


gravity_free_acc_z    0.010684
rot_w                -0.078406
euler_x               0.005358
tof_5                -0.100770
tof_2                -0.031076
gravity_free_acc_x   -0.115861
acc_z                 0.024784
euler_z               0.026900
thm_4                 0.119579
thm_2                 0.035227
thm_5                 0.011972
acc_x                -0.112025
thm_1                 0.109699
thm_3                 0.166354
gravity_free_acc_y    0.101992
tof_1                -0.028424
euler_y              -0.116298
rot_z                -0.076639
rot_y                -0.012926
acc_y                 0.061950
tof_4                -0.023269
rot_x                 0.222167
tof_3                -0.082658
dtype: float64

In [ ]:
pd.concat({
    "train": folds[0][0][feature_cols].agg(["mean", "std"]),
    "validation": folds[0][1][feature_cols].agg(["mean", "std"]),
})

gravity_free_acc_z         rot_w       euler_x         tof_5  \
train      mean       -6.861285e-18  3.430643e-18 -9.801836e-19 -2.940551e-18   
           std         1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean        1.068352e-02 -7.840577e-02  5.358021e-03 -1.007702e-01   
           std         9.509305e-01  9.704027e-01  1.007000e+00  1.023462e+00   

                        tof_2  gravity_free_acc_x         acc_z       euler_z  \
train      mean -5.391010e-18        6.861285e-18  9.801836e-18 -8.821652e-18   
           std   1.000000e+00        1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -3.107589e-02       -1.158611e-01  2.478402e-02  2.689956e-02   
           std   1.009105e+00        9.915675e-01  1.030467e+00  9.750295e-01   

                        thm_4         thm_2  ...         thm_3  \
train      mean  4.900918e-18  5.881101e-18  ...  5.881101e-18   
           std   1.000000e+00  1.000000e+00  ...  1.000000e+00   
validation mean  1.195792e-01  3.522733e-02  ...  1.663541e-01   
           std   5.753183e-01  9.938025e-01  ...  4.822490e-01   

                 gravity_free_acc_y         tof_1       euler_y         rot_z  \
train      mean       -3.920734e-18 -3.920734e-18  5.391010e-18  5.881101e-18   
           std         1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean        1.019922e-01 -2.842408e-02 -1.162984e-01 -7.663910e-02   
           std         1.030047e+00  9.940576e-01  9.731178e-01  1.028713e+00   

                        rot_y         acc_y         tof_4         rot_x  \
train      mean  5.881101e-18 -9.311744e-18  6.861285e-18  4.410826e-18   
           std   1.000000e+00  1.000000e+00  1.000000e+00  1.000000e+00   
validation mean -1.292626e-02  6.194959e-02 -2.326864e-02  2.221670e-01   
           std   9.530785e-01  9.909236e-01  9.848455e-01  1.071349e+00   

                        tof_3  
train      mean  3.675688e-18  
           std   1.000000e+00  
validation mean -8.265772e-02  
           std   9.982566e-01  

[4 rows x 23 columns]

### Normalize sequences lengths.  
And turn the Dataframes into ndarrays.

#### Visualize histogram of sequences lengths.

Entire dataset sequences lengths.

In [ ]:
px.histogram(
    (
        df
        .groupby("sequence_id", observed=True)
        .size()
    ),
    title="Sequence length frequency",
)

Second(to avoid always look at the first one) Train/validation split sequences lengths comparaison.

In [ ]:
def get_set_sequences_lengths(set:DF, name:str) -> DF:
    return (
        set
        .groupby("sequence_id", observed=True)
        .size()
        .reset_index(name="length")
        .assign(set=name)
    )

full_se_lengths = pd.concat((
    get_set_sequences_lengths(folds[2][0], "Train"),
    get_set_sequences_lengths(folds[2][1], "Validation"),
))

fig = px.histogram(
    full_se_lengths,
    x="length",
    color="set",
    barmode="overlay",  # or 'group' if you want side-by-side bars
    nbins=50,           # adjust bin size if needed
    title="Sequence Length Distribution: Train vs Validation"
)

fig.update_traces(opacity=0.8)  # better visibility with overlay
fig.show()


In [ ]:
for train, val in folds:
    print("train normed sequence len:", int(train.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print("validation normed sequence len:", int(val.groupby("sequence_id", observed=True).size().quantile(SEQUENCE_NORMED_LEN_QUANTILE)))
    print()

train normed sequence len: 125
validation normed sequence len: 130

train normed sequence len: 126
validation normed sequence len: 127

train normed sequence len: 131
validation normed sequence len: 114

train normed sequence len: 125
validation normed sequence len: 135

train normed sequence len: 125
validation normed sequence len: 136



#### Sequence length norm implementation

In [ ]:
gesture_cols = df["gesture"].unique()

def length_normed_sequence_feat_arr(sequence: DF, normed_sequence_len: int) -> ndarray:
    features = (
        sequence
        .loc[:, feature_cols]
        .values
    )
    len_diff = abs(normed_sequence_len - len(features))
    if len(features) < normed_sequence_len:
        padded_features = np.pad(
            features,
            ((len_diff // 2 + len_diff % 2, len_diff // 2), (0, 0)),
        )
        return padded_features
    elif len(features) > normed_sequence_len:
        return features[len_diff // 2:-len_diff // 2]
    else:
        return features

def df_to_ndarrays(df:DF, normed_sequence_len:int) -> tuple[np.ndarray, np.ndarray]:
    sequence_it = df.groupby("sequence_id", observed=True, as_index=False)
    x = np.empty(
        shape=(len(sequence_it), normed_sequence_len, len(feature_cols)),
        dtype="float32"
    )
    y = np.empty(
        shape=(len(sequence_it), df["gesture"].nunique()),
        dtype="float32"
    )
    for sequence_idx, (_, sequence) in tqdm(enumerate(sequence_it), total=len(sequence_it)):
        normed_seq_feat_arr = length_normed_sequence_feat_arr(sequence, normed_sequence_len)
        x[sequence_idx] = normed_seq_feat_arr
        # Take the first value as they are(or at least should be) all the same in a single sequence
        y[sequence_idx] = sequence[gesture_cols].iloc[0].values

    return x, y

def get_normed_seq_len(dataset:DF) -> int:
    return int(
        dataset
        .groupby("sequence_id", observed=True)
        .size()
        .quantile(SEQUENCE_NORMED_LEN_QUANTILE)
    )

def fold_dfs_to_ndarrays(train:DF, validation:DF) -> tuple[ndarray, ndarray, ndarray, ndarray]:
    """
    Returns:
        (train X, train Y, validation X, validation Y)
    """
    normed_sequence_len = get_normed_seq_len(train)
    return (
        *df_to_ndarrays(train, normed_sequence_len),
        *df_to_ndarrays(validation, normed_sequence_len),
    )

folds_arrs = list(starmap(fold_dfs_to_ndarrays, folds))

  0%|          | 0/6567 [00:00<?, ?it/s]

  0%|          | 0/1584 [00:00<?, ?it/s]

  0%|          | 0/6576 [00:00<?, ?it/s]

  0%|          | 0/1575 [00:00<?, ?it/s]

  0%|          | 0/6519 [00:00<?, ?it/s]

  0%|          | 0/1632 [00:00<?, ?it/s]

  0%|          | 0/6576 [00:00<?, ?it/s]

  0%|          | 0/1575 [00:00<?, ?it/s]

  0%|          | 0/6571 [00:00<?, ?it/s]

  0%|          | 0/1580 [00:00<?, ?it/s]

In [ ]:
full_dataset_sequence_length_norm = get_normed_seq_len(df)
full_x, full_y = df_to_ndarrays(df, full_dataset_sequence_length_norm)

  0%|          | 0/8151 [00:00<?, ?it/s]

## Create dataset

In [ ]:
# Clean dataset directory if it already exists
! rm -rf preprocessed_dataset
# Create dataset direcory
! mkdir preprocessed_dataset
# Save folds
for fold_i, (train_x, train_y, val_x, val_y) in enumerate(folds_arrs):
    fold_dir_path = join("preprocessed_dataset", f"fold_{fold_i}")
    os.makedirs(fold_dir_path)
    # save features (X)
    np.save(join(fold_dir_path, "train_X.npy"), train_x, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_X.npy"), val_x, allow_pickle=False)
    # Save targets (Y)
    np.save(join(fold_dir_path, "train_Y.npy"), train_y, allow_pickle=False)
    np.save(join(fold_dir_path, "validation_Y.npy"), val_y, allow_pickle=False)
# Save full dataset
full_dataset_dir_path = "preprocessed_dataset/full_dataset"
os.makedirs(full_dataset_dir_path)
np.save(join(full_dataset_dir_path, "X.npy"), full_x, allow_pickle=False)
np.save(join(full_dataset_dir_path, "Y.npy"), full_y, allow_pickle=False)
# Save dataset meta data
full_dataset_meta_data["target_names"] = one_hot_target.columns.to_list()
full_dataset_meta_data["pad_seq_len"] = full_dataset_sequence_length_norm
full_dataset_meta_data["feature_cols"] = feature_cols

with open("preprocessed_dataset/full_dataset_meta_data.json", "w") as fp:
    json.dump(full_dataset_meta_data, fp, indent=4)

In [ ]:
df[feature_cols]

,gravity_free_acc_z,rot_w,euler_x,tof_5,tof_2,gravity_free_acc_x,acc_z,euler_z,thm_4,thm_2,...,thm_3,gravity_free_acc_y,tof_1,euler_y,rot_z,rot_y,acc_y,tof_4,rot_x,tof_3
0,-0.191143,-0.997151,0.191240,0.404678,0.148727,0.921929,0.625816,0.339185,0.367246,1.184774,...,0.633442,0.378283,0.339571,0.883372,-1.238540,-0.716343,0.884130,-0.016778,-0.508525,-0.454938
1,-0.239701,-0.957737,0.150843,0.433771,0.199531,0.935808,0.588012,0.297587,0.386687,1.197089,...,0.681437,0.441917,0.349948,0.860138,-1.267920,-0.681840,0.884130,0.002596,-0.476437,-0.337148
2,0.018062,-0.630264,-0.047236,0.490583,0.371769,0.779645,0.964766,0.140441,0.423273,0.976524,...,0.741405,0.663317,0.398864,0.754947,-1.349747,-0.549352,0.723320,0.026308,-0.334147,0.022423
3,0.236127,-0.290093,-0.231340,0.532300,0.644067,0.839486,1.134561,-0.051374,0.484648,0.063125,...,0.584725,0.700636,0.621508,0.714206,-1.390175,-0.329928,0.347835,0.459197,-0.312448,0.543179
4,0.639598,-0.134027,-0.342103,0.684347,0.789974,0.600054,1.655481,-0.252579,0.592139,-0.353110,...,0.081696,0.487647,1.073910,0.418963,-1.447599,-0.007333,-0.302433,0.731018,-0.214344,0.870276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574940,-0.166781,-1.117506,1.457859,-1.044439,-0.829872,0.345242,-1.309212,-1.902655,0.854270,0.279611,...,0.292463,-1.541394,-1.001627,0.287935,0.070326,-0.759754,-0.444509,-0.929112,-1.601608,0.541703
574941,-0.249065,-1.100313,1.471826,-1.014798,-0.851557,0.390242,-1.434796,-1.902869,0.870208,0.311259,...,0.314339,-1.547135,-0.969906,0.326326,0.045559,-0.740360,-0.490566,-0.996778,-1.607920,0.533437
574942,-0.024379,-1.089203,1.470549,-0.963200,-0.925905,0.333282,-1.138775,-1.901584,0.848956,0.301660,...,0.299698,-1.418637,-1.037499,0.335326,0.041674,-0.741939,-0.314143,-1.002272,-1.605422,0.399410
574943,-0.223986,-1.070422,1.484620,-1.007662,-0.894307,0.433050,-1.414933,-1.901015,0.864896,0.389432,...,0.330572,-1.473273,-1.027123,0.374099,0.017150,-0.722432,-0.429677,-0.900484,-1.610813,0.467900


## Dataset upload
Optionally upload the dataset to kaggle.

In [ ]:
if input("Do you want to upload the  dataset to kaggle?[yes/no]").lower() == "yes":
    # Updaload the dataset
    dataset_upload(
        join(whoami()["username"], "prepocessed-cmi-2025"),
        "preprocessed_dataset",
        version_notes="Preprocessed Child Mind Institue 2025 competition dataset."
    )
else:
    print("Dataset has not been uploaded.")

Dataset has not been uploaded.
